# Generating the required sample size for the Push Notifications

## Using the Benjamini-Hochberg for estimation

### Benjamini-Hochberg

In [1]:
import numpy as np 
import pandas as pd
import scipy as stats
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection

In [2]:
### Fixed Parametrers ###
n_days = 21
n_slots = 15
baseline_ctr = 0.3 # Base line click Through Rate
lift = 0.0007 # (20%)
alpha_fdr = 0.05
target_power = 0.80
reps = 200 # Reps for Monte Carlo
###################

# Lock the randomisation: 
rng = np.random.default_rng(42)

In [3]:
### Simulating Function ###

def run_one_simulation(n_users: int) -> bool:
    
    slots = rng.integers(0, n_slots, size = (n_users, n_days))

    # True Conversion Rate: 
    true_CTR = np.full(n_slots, baseline_ctr)
    true_CTR[8] += lift # Setting that slot 8 performs better

    # Simulate Clicks (Bernoulli outcomes)
    clicks =  rng.binomial(1, true_CTR[slots])

    # To DF
    df = pd.DataFrame({
        "slot": slots.ravel(),
        "click": clicks.ravel()
    })

    ### Computing data per slot ###
    groups_stats = df.groupby("slot")["click"].agg(["mean", "count"])
    baseline_p = groups_stats.loc[0,"mean"]
    baseline_n = groups_stats.loc[0,"count"]

    ### Two-proportion z-test vs baseline slot (slot 0) ###
    p_values = []
    for s in range(n_slots):
        p = groups_stats.loc[s, "mean"]
        n = groups_stats.loc[s, "count"]

        p_comb = (p*n + baseline_p*baseline_n) / (n + baseline_n)
        se = np.sqrt(p_comb * (1 - p_comb) * (1/n + 1/baseline_n))
        z = (p - baseline_p) / se
        pval = 2 * (1 - stats.norm.cdf(abs(z)))
        p_values.append(pval)

    # Apply Benjamin-Hochberg FDR correction
    reject, _ = fdrcorrection(p_values, alpha = alpha_fdr, method = 'indep')

    # Did we detect the true best slot? (In this case we set at 8)
    return reject[8]

In [4]:
### Function: Estimating Power ###
def estimate_power(n_users: int) -> float:
    detections = sum(run_one_simulation(n_users) for _ in range(reps))
    return detections / reps

In [5]:
### Going through different Ns ###
candidate_ns = [145000]
results = []
for n in candidate_ns:
    power = estimate_power(n)
    results.append((n, power))
    print(f"n_users = {n:7,d} -> estimated power = {power:.2f}")

n_users = 145,000 -> estimated power = 0.03


### Finding best min N

In [7]:
df_power = pd.DataFrame(results, columns = ["n_users", "power"])
best = df_power[df_power["power"] >= target_power].head(1)

if not best.empty:
    best_n = int(best.iloc[0,0])
    print(f"\n✅ Smallest n achieving ≥{target_power:.0%} power: {best_n:,} users")
else:
    print("\n⚠️ No tested n reached target power — try increasing range.")

print("\nFull results:\n", df_power)


⚠️ No tested n reached target power — try increasing range.

Full results:
    n_users  power
0   145000  0.025
